In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import pipeline

/home/apprenant/miniconda3/envs/nlp_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_csv('../data/Emotion_final.csv')

In [5]:
label_encoder = LabelEncoder()

X = df['Text'].to_list()
y = label_encoder.fit_transform(df['Emotion'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

==============================================================

In [6]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

**from transformers import DistilBertTokenizerFast** : Cette ligne importe la classe DistilBertTokenizerFast à partir du module transformers.

**tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')** : Cette ligne crée une instance du tokenizer en utilisant la méthode from_pretrained pour charger le modèle pré-entraîné **"distilbert-base-uncased"**. "distilbert-base-uncased" est un modèle de DistilBERT qui a été entraîné sur des textes en anglais en minuscules.

ce code charge un tokenizer basé sur le modèle DistilBERT pré-entraîné, ce qui vous permet de prétraiter le texte de manière efficace pour une utilisation ultérieure, par exemple, pour effectuer de la classification de texte ou de la génération de texte en utilisant le modèle DistilBERT.

In [7]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

**train_encodings = tokenizer(X_train, truncation=True, padding=True)**: Cette ligne utilise le tokenizer pour encoder **(tokenizer)** les données d'entraînement **(X_train)**. Les options **truncation=True** et **padding=True** indiquent que le texte doit être tronqué (s'il dépasse une certaine longueur) et que du padding (remplissage) doit être ajouté aux séquences plus courtes pour les rendre de longueur égale. La variable **train_encodings** stocke les résultats de l'encodage.

**test_encodings = tokenizer(X_test, truncation=True, padding=True)**: Cette ligne fait la même chose que la précédente, mais pour les données de test **(X_test)**. Les options de troncature et de padding sont les mêmes.

In [8]:
import torch

class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmotionDataset(train_encodings, y_train)
test_dataset = EmotionDataset(test_encodings, y_test)

In [9]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

## Partager Model

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("./results/checkpoint-2500")

In [ ]:
model = DistilBertForSequenceClassification(config=config)

In [ ]:
tokenizer.save_pretrained("outputs/tokiniser")

In [ ]:
model.base_model.save_pretrained("outputs/model")

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.push_to_hub("Charles-59800/my-awesome-model")

In [ ]:
tokenizer.push_to_hub("my-awesome-model")